In [38]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

# Q1. Downloading the data

We'll use the NYC taxi dataset, specifically we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023. Read the data for January. How many columns are there?

In [26]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
print(f'Columns in file: {df.shape[1]}')

Columns in file: 19


In [27]:
df.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0


# Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

In [28]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)
print(f'The standard deviation is: {df.duration.std()}')

The standard deviation is: 42.594351241920904


# Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [29]:
original_shape = df.shape[0]
df = df[(df.duration >= 1) & (df.duration <= 60)]
new_shape = df.shape[0]

In [30]:
print(f'The fraction left is: {new_shape / original_shape * 100}')

The fraction left is: 98.1220282212598


# Q4. One-hot encoding

Let's apply one-hot encodicategorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']ng to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [31]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [32]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [35]:
print(f'The number of columns is: {X_train.shape[1]}')

The number of columns is: 515


# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [46]:
y_train = df['duration']
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print(f'RMSE on training data is: {root_mean_squared_error(y_train, y_pred)}')

RMSE on training data is: 7.649261927686161


# Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).
What's the RMSE on validation?

In [49]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda x: x.total_seconds() / 60)
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)] # Required by the task
df_val[categorical] = df_val[categorical].astype(str)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [50]:
y_val = df_val['duration']
val_pred = lr.predict(X_val)

print(f'RMSE on validation data is: {root_mean_squared_error(y_val, val_pred)}')

RMSE on validation data is: 7.811817957524739
